## My custom design notes
* For the end-caps for the rods use crush ribs. 
  * The hole should be drafted at about 2 degrees, the ribs should not be drafted. 
  * The ribs should interfer with the rod by .254 mm and that is the dimensions of the radius of it. 
  * The bottom of the draft hole is the same dimension as the rod. 

## Motor sizing calculations:

### Sympy printing function and setup

In [398]:
from sympy import *
from IPython.display import display, Math

def printEvaluation(expr, values, var_name, unit=''):
    """Printing funciton for sympy expressions to show full expression and calculation

    Parameters
    ----------
    expr
        A sympy expression to be displayed and evaluated
    values : dict
        A dictionary containing each of the varables in the expression as keys and their values as the ...values
    var_name : str
        Just the str version of the expression's name (the left side of the equation)
    unit : str
        (optional) The engineering units of the answer (e.g. N, kg, etc.)
    Returns
    -------
    The calculated value

    Example
    -------
    theta, y = symbols('theta, y')
    g = cos(theta / y)
    values = {theta: 1, y: 4}
    printEvaluation(g, values, 'g')
    """

    # Make an alternative dict with values that won't be evaluated. 
    uneval_vals = {}
    for key, value in values.items():
        uneval_vals[key] = UnevaluatedExpr(value)
    result = expr.evalf(subs=values)
    display(Math(f'{var_name} = {latex(nsimplify(expr))} = {latex(expr.subs(uneval_vals))} = {result:.4f} \\text{{ }} {unit}'))

    return result

### First reference calculations

In [399]:
# From: https://medium.com/husarion-blog/10-steps-to-choosing-the-right-motors-for-your-robotic-project-bf5c4b997407
# Inputs (everything in SI units): 

m_R, v_n, D_W, t_A, k, g = symbols("m_R, v_n, D_W, t_A, k, g")
values = {m_R: .907,  # Mass of the robot in kg
          v_n: 2.2,   # Linear speed of the robot in m/s
          D_W: 0.143, # Diameter of wheel in m
          t_A: 0.5,   # Time it takes to get to full speed in s 
          k:   0.57,  # Percent grade in %  
          g:   9.81}  # gravity in m/s^2

# .907 kg is 2 lbs
# 2.2 m/s is 5 mph
# .57 slope is 30 degrees

In [400]:
# Wheel rotation speed [RPM]
N_T = (60*v_n)/(pi*D_W)
N_Tval = printEvaluation(N_T, values, 'N_T', 'rpm')

<IPython.core.display.Math object>

In [401]:
# Thrust force for whole robot to get up a hill [N]
F_T = g*k*m_R
F_Tval = printEvaluation(F_T, values, 'F_T', 'N')
F_T = symbols('F_T')
values[F_T] = F_Tval

<IPython.core.display.Math object>

In [402]:
# Mechanical power for both motors together [W]
P_U = F_T*v_n
P_Uval = printEvaluation(P_U, values, 'P_U', 'W') 

<IPython.core.display.Math object>

In [403]:
# Torque per motor [Nm]
T_T = (D_W/2)*F_T  # The example did per wheel but each of our motors drives 2 wheels. 
T_Tval = printEvaluation(T_T, values, 'T_T', 'Nm')

<IPython.core.display.Math object>

In [404]:
# Vehicle acceleration 
A_R = v_n / t_A
A_Rval = printEvaluation(A_R, values, 'A_R', 'm/s')
A_R = symbols("A_R")
values[A_R] = A_Rval

<IPython.core.display.Math object>

In [405]:
# Acceleration force
F_A = A_R*m_R
F_Aval = printEvaluation(F_A, values, 'F_A', 'N')
# Since this is less that F_T I'm not going to use it. 

<IPython.core.display.Math object>

### Second reference calculations

In [406]:
# From https://wiki.dfrobot.com/How_to_Calculate_the_Motor_Torque_for_a_Mobile_Robot

# Next ones are: 
# https://www.robotsforroboticists.com/motor-selection/
# https://maker.pro/custom/tutorial/motor-sizing-math